In [0]:
%python
# Define the path to the CSV file
csv_file_path = "/Volumes/workspace/globalretail/customer"

# Read the CSV file into a DataFrame
customer_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)

# Display the DataFrame
display(customer_df)